In [71]:
# 設定日期區間

import datetime

def dt_to_str(date_obj):
    return '{:04}'.format(date_obj.year)+'-'+'{:02}'.format(date_obj.month)+'-'+'{:02}'.format(date_obj.day)

def str_to_dt(string):
    word = string.split('-')
    y = int(word[0])
    m = int(word[1])
    d = int(word[2])
    return datetime.date(y, m, d)

start = datetime.date(2022, 3, 29)
end = datetime.date(2022, 5, 13)
time = start
lst = []

#donnot_get = ['2022-05-01']

while time <= (end + datetime.timedelta(1)):
    lst += [time]
    time += datetime.timedelta(1)

#print(lst)

days = [dt_to_str(d) for d in lst]

#print(days)

In [80]:
# 處理車次列表 (現成檔案)
import json

with open('TRATimeTable_2022-03-29_0301.json', 'r', encoding='utf8') as f:
    json_file = json.load(f)

with open('Exclude.txt', 'r', encoding='utf8') as f:
    exclude = []
    lines = f.readlines()
    for line in lines:
        try:
            data = line.strip().split('：')[-1].split(chr(9))  #拆分車次資料
            for d in data:
                exclude += [d]
        except:
            break

ids = []

for train in json_file["TrainTimetables"]:
    i = train["GeneralTrainInfo"]["TrainNo"]
    if i not in exclude and i[-1].isalpha() == False:
        ids += [i]

In [ ]:
print(len(ids))
print(exclude)

In [ ]:
# 抓取歷史紀錄 (現成檔案)
import csv
from tqdm.notebook import tqdm
database = []
for d, tq in zip(days, tqdm(days)):
    route = f'.\\DailyDelayInfo\\{d}.csv'
    with open(route, 'r', encoding='utf8') as csv_file:
        whole_file = list(csv.reader(csv_file))
        title = whole_file[0]
    database += [[d[0], d[2], d[4], d[5]] for d in whole_file[1:]]
days = days[:-1]  #將原本多加進來的一天去除

In [77]:
# 讀取站名資料 (現成檔案)
with open('Stations.txt', 'r', encoding='utf8') as f:
    sta_lst = f.readlines()
    sta_lst = [i.strip() for i in sta_lst]

In [82]:
# 讀取、轉換資料

import datetime
from pprint import pprint
import json
import csv
from tqdm.notebook import tqdm


#ids = ['666']

def time(string):
    if string[-2:] == '.5':
        second = 30
        string = string.replace('.5', '')
    else:
        second = 0
    hour = int(string.split(':')[0])
    minute = int(string.split(':')[1])
    return datetime.time(hour,minute,second)

#for ID in ids:
for ID, tq in zip(ids, tqdm(ids)):
    for i, train in enumerate(json_file["TrainTimetables"]):
        if train["GeneralTrainInfo"]["TrainNo"] == ID:
            for sta in train["Stations"]:
                if sta["StationName"] == (train["GeneralTrainInfo"]["StartingStationName"] if train["GeneralTrainInfo"]["StartingServiceStationName"] == '' else train["GeneralTrainInfo"]["StartingServiceStationName"]):
                    start_time = time(sta["DepartureTime"])
                    break
            sta_record_order = []
            for k in train["Stations"]:
                if k["StationName"] in sta_lst:
                    sta_record_order += [k["StationName"]]
            
            sta_record_order = sta_record_order[:-1]

            # 刪除原本用json資料處理起終點與環島遇到重複站的內容
            # 注意！因為紀錄只到倒數第二站，sta_record_order 直接去掉最後一站
    
    delay_data_lst = []
    this_train_data = []

    for line in database:   #挑出當次車的資料
        if line[0] == ID:
            this_train_data += [{'StationNameZh_tw':line[1], \
                                'DelayTime':int(line[2]), \
                                'SrcUpdateTime':line[3]}]

    def record_time_transfer(string):
        return datetime.datetime.strptime(string, "%Y-%m-%d %H:%M:%S%z")

    this_train_data.sort(key=lambda ttd: record_time_transfer(ttd['SrcUpdateTime']))
    daily_data = {}

    for day in days:      #挑出各日期的資料
        for i, ttd in enumerate(this_train_data):
            the_date_of_record = record_time_transfer(ttd['SrcUpdateTime']).date()
            dt_day = datetime.datetime.strptime(day, "%Y-%m-%d").date()
            record_time = record_time_transfer(ttd['SrcUpdateTime'])
            record_time = record_time.replace(tzinfo=None)
            if the_date_of_record == dt_day and \
                record_time >= datetime.datetime.combine(dt_day, start_time):
                daily_data[day] = []
                ind = i
                while True:
                    daily_data[day] += [this_train_data[ind]]
                    #now_time = record_time_transfer(this_train_data[i]['SrcUpdateTime'])
                    if ind+1 == len(this_train_data):
                        break
                    next_time = record_time_transfer(this_train_data[ind+1]['SrcUpdateTime'])
                    next_time = next_time.replace(tzinfo=None)
                    if next_time >= datetime.datetime.combine(dt_day+datetime.timedelta(1), start_time):
                        break
                    else:
                        ind += 1
                break

    if len(daily_data) == 0:
        print(f'No data in TrainNo {ID}')
        continue

    repeat_data_removed = {}

    for day in days:     #依站序抓取資料
        if day not in daily_data:
            continue
        tmp_data = []
        for sta in sta_record_order:
            record = []
            for data in daily_data[day]:
                if data['StationNameZh_tw'] == sta:
                    record += [data]
            delay_time = -1   #初始值、異常值
            if len(record) == 0:
                tmp_data += [{'StationNameZh_tw':sta, \
                                'DelayTime':-1, \
                                'SrcUpdateTime':""}]
            else:
                for r in record:
                    if r['DelayTime'] > delay_time:
                        this_sta_data = r
                        delay_time = r['DelayTime']
                tmp_data += [r]
        repeat_data_removed[day] = tmp_data

    if True:
        with open(f'.\\Cleaned\\{ID}.json','w',encoding='utf8') as f:
            json.dump(repeat_data_removed, f, indent=4, ensure_ascii=False)
    else:
        pprint(repeat_data_removed)


  0%|          | 0/683 [00:00<?, ?it/s]